In [339]:
import tensorflow as tf
from keras.models import Model, load_model
import keras.backend as K
from keras import layers, models, utils
import keras
import numpy as np
import os
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
import re
import random
import pandas as pd
import cv2
import math
import datetime
print(tf.version.VERSION)

2.2.0


In [340]:
stock_data = pd.read_excel('10StockPortfolio.xlsx')
print(stock_data.columns)

Index(['Date', 'Netflix', 'Google', 'Amazon', 'American Airlines',
       'Bank of America', 'Danaher', 'Ford', 'Nike', 'Apple', 'BP', 'S&P 500'],
      dtype='object')


In [0]:
data = []
date_to_num = {}
num_to_date = {}
i = 0
for row in stock_data.iterrows():
    datapoint = []
    for column in stock_data.columns:
        if column != 'Date' and column != 'S&P 500':
            datapoint.append(row[1][column])
    data.append(datapoint)
    if i >= 100:
        date_to_num.update({row[1]['Date'] : i-100})
        num_to_date.update({str(i-100): row[1]['Date']})
    i = i+1

In [342]:
data = np.array(data,dtype=np.float32)
print(data.shape)

(2606, 10)


In [343]:
last100days = np.zeros((data.shape[0]-101,data.shape[1],100))
variances = np.zeros((data.shape[0]-101,10,1))
covariances = np.zeros((data.shape[0]-101,10,10))
percent_returns = np.zeros((data.shape[0]-101,10,1))
for i in range(100,data.shape[0]-1):
    last100days[i-100] = np.swapaxes(data[i-100:i],0,1)
    variances[i-100] = np.var(data[i-100:i],axis=0)[:,np.newaxis]
    covariances[i-100] = np.cov(data[i-100:i],rowvar=False)[:,:]
    percent_returns[i-100] = (((data[i+1] / data[i]) - 1) * 100)[:,np.newaxis]
    ##percent returns is the returns on the stock for the next day
print(last100days.shape)
print(variances.shape)
print(percent_returns.shape)
print(covariances.shape)
    

(2505, 10, 100)
(2505, 10, 1)
(2505, 10, 1)
(2505, 10, 10)


In [344]:
print(num_to_date[str(1158)])
print(num_to_date[str(1410)])
##Validate on 1158:1410
print(num_to_date[str(2164)])
##Test on 2164:
##Train on 0:1158 and 1410:2164

##Subtract 100 from all numbers


2015-01-02 00:00:00
2016-01-04 00:00:00
2019-01-02 00:00:00


In [0]:
last100days_layer = layers.Input(shape=(10,100))
percent_returns_layer = layers.Input(shape=(10,1))
covariances_layer = layers.Input(shape=(10,10))

x = layers.Conv1D(1024,1)(last100days_layer)
x = layers.LeakyReLU()(x)
x = layers.Conv1D(512,1)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv1D(128,1)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv1D(32,1)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv1D(1,1)(x)
portfolio = layers.Softmax(axis=1)(x)

## Compute mean returns for the day for portfolio
returns = layers.Multiply()([portfolio,percent_returns_layer])
returns_num = layers.Lambda(lambda xin: K.sum(xin, axis=1))(returns)
returns = layers.Activation(keras.activations.sigmoid)(returns_num)

##Compute volatility of the portfolio using covariance matrix
portfolio_transpose = layers.Lambda(lambda xin: K.permute_dimensions(xin,(0,2,1)))(portfolio)
covariance = layers.Dot(axes=(2,1))([portfolio_transpose,covariances_layer])
covariance = layers.Dot(axes=(2,1))([covariance,portfolio])
covariance = layers.Lambda(lambda xin: K.sqrt(xin))(covariance)

volatility_num = layers.Lambda(lambda xin: K.squeeze(xin,axis=-1))(covariance)
volatility = layers.Activation(keras.activations.tanh)(volatility_num)






In [0]:
optimizer = tf.keras.optimizers.Adam(
    learning_rate=0.000001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,
    name='Adam'
)

In [0]:
loss = tf.keras.losses.BinaryCrossentropy(
    from_logits=False,
    name='binary_crossentropy'
)

def maxWeight(y_true, y_pred, HUBER_DELTA = 0.125):
   x   = K.abs(y_true - y_pred)
   x   = K.switch(x < HUBER_DELTA, 0*x, 2*(x-HUBER_DELTA) **2)
   return  K.sum(x)

In [0]:
weight_constraint = np.zeros((variances.shape[0],10,1)) + 0.175
true = np.zeros((variances.shape[0],2))
true[:,0] = 1
true[:,1] = 0

In [0]:
network = Model(inputs=[last100days_layer,percent_returns_layer,covariances_layer],outputs=[portfolio,returns,volatility])

In [360]:
network.compile(optimizer=optimizer,loss=[maxWeight,loss,loss])
network.summary()

Model: "model_44"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_170 (InputLayer)          (None, 10, 100)      0                                            
__________________________________________________________________________________________________
conv1d_213 (Conv1D)             (None, 10, 1024)     103424      input_170[0][0]                  
__________________________________________________________________________________________________
leaky_re_lu_170 (LeakyReLU)     (None, 10, 1024)     0           conv1d_213[0][0]                 
__________________________________________________________________________________________________
conv1d_214 (Conv1D)             (None, 10, 512)      524800      leaky_re_lu_170[0][0]            
___________________________________________________________________________________________

In [361]:
network.fit(x=[last100days[:2000],percent_returns[:2000],covariances[:2000]],y=[weight_constraint[:2000],true[:2000,0],true[:2000,1]],epochs=20,batch_size=10
            ,validation_data=[
                             [last100days[2000:2064],percent_returns[2000:2064],covariances[2000:2064]],
                             [weight_constraint[2000:2064],true[2000:2064,0],true[2000:2064,1]]
            ]
            )

Train on 2000 samples, validate on 64 samples
Epoch 1/20
2000/2000 [==============================] - 6s 3ms/step - loss: 2.4178 - softmax_43_loss: 0.3506 - activation_63_loss: 0.8479 - activation_64_loss: 1.2192 - val_loss: 2.3460 - val_softmax_43_loss: 0.8180 - val_activation_63_loss: 0.7650 - val_activation_64_loss: 0.6905
Epoch 2/20
2000/2000 [==============================] - 6s 3ms/step - loss: 2.1372 - softmax_43_loss: 0.2178 - activation_63_loss: 0.8336 - activation_64_loss: 1.0859 - val_loss: 2.4236 - val_softmax_43_loss: 0.9199 - val_activation_63_loss: 0.7689 - val_activation_64_loss: 0.6476
Epoch 3/20
2000/2000 [==============================] - 6s 3ms/step - loss: 2.1049 - softmax_43_loss: 0.2164 - activation_63_loss: 0.8312 - activation_64_loss: 1.0572 - val_loss: 2.1367 - val_softmax_43_loss: 0.6237 - val_activation_63_loss: 0.7611 - val_activation_64_loss: 0.6862
Epoch 4/20
2000/2000 [==============================] - 6s 3ms/step - loss: 2.0828 - softmax_43_loss: 0.2156

In [365]:
network.save("10StockPortfolioModel.h5")

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:165: UserWarning: TensorFlow optimizers do not make it possible to access optimizer attributes or optimizer state after instantiation. As a result, we cannot save the optimizer as part of the model save file.You will have to compile your model again after loading it. Prefer using a Keras optimizer instead (see keras.io/optimizers).
  'TensorFlow optimizers do not '


In [0]:
portfolio_model = Model(inputs=last100days_layer,
                                 outputs=portfolio)

In [0]:
predictions = portfolio_model.predict(last100days[:2000])

In [396]:
print("Example Portfolio Day:")
print(predictions[900])

Example Portfolio Day:
[[8.0370326e-03]
 [2.2480667e-15]
 [1.0794277e-13]
 [3.0913968e-02]
 [2.8372556e-01]
 [1.2692270e-01]
 [3.2336596e-01]
 [7.1913831e-02]
 [2.9182713e-02]
 [1.2593819e-01]]


In [0]:
test_model = Model(inputs=[last100days_layer,percent_returns_layer,covariances_layer],outputs=[portfolio,returns_num,volatility_num])

In [0]:
train_predictions = test_model.predict(x=[last100days[:2000],percent_returns[:2000],covariances[:2000]])

In [393]:
print("Percent Return on Training Data:")
print(np.mean(train_predictions[1]*252))
print("Average Volatility of Portfolio on Training Data:")
print(np.sqrt(np.mean(train_predictions[2])))

Percent Return on Training Data:
18.164627
Average Volatility of Portfolio on Training Data:
0.8529321


In [0]:
test_predictions = test_model.predict(x=[last100days[2064:],percent_returns[2064:],covariances[2064:]])

In [395]:
print("Percent Return on Testing Data:")
print(np.mean(test_predictions[1]*252))
print("Average Volatility of Portfolio on Testing Data:")
print(np.sqrt(np.mean(test_predictions[2])))

Percent Return on Testing Data:
-26.139975
Average Volatility of Portfolio on Testing Data:
1.0296574
